In [27]:
%matplotlib tk
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
import scipy.io as sio
import h5py
import time as time
import scipy.optimize as opt
import os
import scipy
from tqdm import tqdm
import scipy.ndimage
from matplotlib import colors

import parmap
import glob
from scipy.spatial import distance
import cv2

import torch
from torch import nn
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def svd_denoise(rank, u, Vs, fname):
    denoised=(u[:,:rank] @ Vs[:rank].transpose(1,2,0)[:,:,:,None]).squeeze()

    ctr=0
    for k in range(0,5000,500):
        ax=plt.subplot(2,5,ctr+1)
        plt.imshow(denoised[:,:,k],vmin=-50.0,vmax=50.0)
        ctr+=1
    plt.show()
    
    root_dir = os.path.split(fname)[0]
    np.save(root_dir + '/denoised.npy', denoised)

    return denoised

def svd_gpu(data, fname):
    
    data_in = data#.transpose(2,0,1)#.transpose(2,0,1)
    print ("Data in: ", data_in.shape)
    data_in = np.float32(data_in)
    data_in = data_in.reshape(data_in.shape[0],-1)
    print (" Data going into GPU SVD: ", data_in.shape)

    root_dir = os.path.split(fname)[0]
    
    if os.path.exists(root_dir+'/v_gpu.pt')==False:

        a = torch.from_numpy(np.float32(data_in)).float().to(device)
        print ("Input array size: ", a.shape)
        print ("...starting GPU-svd")
        u, s, v = torch.svd(a)

        print ("u: ", u.shape)
        print ("s: ", s.shape)
        print ("v: ", v.shape)

        # scale temporal component by singular vals

        torch.save(v,root_dir+'/v_gpu.pt')
        torch.save(s,root_dir+'/s_gpu.pt')
        torch.save(u,root_dir+'/u_gpu.pt')

        v = v.cpu().data.numpy()
        s = s.cpu().data.numpy()
        u = u.cpu().data.numpy()

        np.save(root_dir+'/v.npy',v)
        np.save(root_dir+'/s.npy',s)
        np.save(root_dir+'/u.npy',u)

    #     Vs = (v*s).transpose(1,0).reshape(-1,128,128)
    #     np.save(root_dir+'Vs.npy',Vs)

    else:
        print ("loading from disk")

        try: 
            v = np.load(root_dir+'/v.npy')
            s = np.load(root_dir+'/s.npy')
            u = np.load(root_dir+'/u.npy')

        except:

            u = torch.load(root_dir+'/u_gpu.pt').cpu().data.numpy()
            s = torch.load(root_dir+'/s_gpu.pt').cpu().data.numpy()
            v = torch.load(root_dir+'/v_gpu.pt').cpu().data.numpy()

            np.save(root_dir+'/v.npy',v)
            np.save(root_dir+'/s.npy',s)
            np.save(root_dir+'/u.npy',u)    

    Vs = (v*s).transpose(1,0).reshape(-1,128,128)
    np.save(root_dir+'/Vs.npy',Vs)

    print ("Vs: ", Vs.shape)
    print ("u: ", u.shape)
    print ("DONE")

    return u, Vs

In [46]:
# OPTIONAL MASK THE DATA
data = np.fromfile('/media/cat/4TBSSD/Downloads/chronic_variable_stress/343M_Baseline2_30Hz_Unfiltered_float32_256-256-9028.raw','float32')
print (data.shape)
size = [256,256]

data2D = data.reshape(-1,size[0],size[1])
print (data2D.shape)


(591659008,)
(9028, 256, 256)


In [58]:
plt.imshow(data2D[1000])
plt.show()


In [48]:
print (data2D[5000])
plt.hist(data2D.ravel())
plt.show()


[[ 8438339    53059  8444995 ...  8442691    51267  8441155]
 [   51267  8444483  8448067 ...    56899    53571    50755]
 [ 8443971    60227    64835 ...    57411    54083  8439875]
 ...
 [12617796    36932  6328132 ...    31812 12609348  8414532]
 [ 8423748  4229956  8424516 ... 12609604  4219204  4216388]
 [ 8423236  4229700 12618052 ...  8413764    23620    21828]]


In [59]:
# video writing step
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
fname_out = '/media/cat/4TBSSD/Downloads/control/351M_Baseline2_30Hz_Unfiltered_float32_256-256-9031.avi'
ctr=0
video_out = cv2.VideoWriter(fname_out,fourcc, 30, (256,256), 1)
#video_out = cv2.VideoWriter(fname_out, cv2.VideoWriter_fourcc(*'PIM1'), 25, (256,256), 1)

for k in range(100):
    frame = np.uint8(data2D[k]-data2D[k].mean(0))
    print (frame.shape)
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    print (frame.shape)
    print (frame[0][:10])
    #frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    
    video_out.write(frame)

video_out.release()
    

(256, 256)
(256, 256, 3)
[[189 189 189]
 [121 121 121]
 [ 38  38  38]
 [  9   9   9]
 [ 44  44  44]
 [ 95  95  95]
 [140 140 140]
 [218 218 218]
 [ 89  89  89]
 [ 14  14  14]]
(256, 256)
(256, 256, 3)
[[173 173 173]
 [148 148 148]
 [ 73  73  73]
 [ 86  86  86]
 [248 248 248]
 [ 51  51  51]
 [ 82  82  82]
 [165 165 165]
 [254 254 254]
 [  9   9   9]]
(256, 256)
(256, 256, 3)
[[ 76  76  76]
 [255 255 255]
 [143 143 143]
 [ 84  84  84]
 [ 74  74  74]
 [ 71  71  71]
 [113 113 113]
 [178 178 178]
 [ 39  39  39]
 [  4   4   4]]
(256, 256)
(256, 256, 3)
[[ 29  29  29]
 [248 248 248]
 [ 92  92  92]
 [ 91  91  91]
 [ 47  47  47]
 [ 45  45  45]
 [125 125 125]
 [169 169 169]
 [253 253 253]
 [ 46  46  46]]
(256, 256)
(256, 256, 3)
[[ 38  38  38]
 [246 246 246]
 [120 120 120]
 [144 144 144]
 [ 34  34  34]
 [ 22  22  22]
 [123 123 123]
 [102 102 102]
 [188 188 188]
 [198 198 198]]
(256, 256)
(256, 256, 3)
[[210 210 210]
 [118 118 118]
 [178 178 178]
 [145 145 145]
 [169 169 169]
 [214 214 214]
 [217

In [4]:
#root_dir = '/media/cat/1TB/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort6_DC2/Cohort6_DC2/Baseline/'
#root_dir = '/media/cat/1TB/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort6_DC2/Cohort6_DC2/Post-CSD/CSD/'
# home = '/media/cat/1TB/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort4_1'
# root_dirs = os.listdir(home)
# print (root_dirs)
# root_dir = root_dirs[1]
# print (root_dir)

# print (home+'/'+root_dir + '/control/control*')
# fname = glob.glob(home+'/'+root_dir + '/control/control*')[1]
# print (fname)

root_dir = '/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_A1/'
fnames = os.listdir(root_dir)
print (fnames)


data = np.load(root_dir + fnames[1]+'/'+'csd_5000frames.npy')
print (data.shape)


fname = '/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_A1/csd/csd_5000frames.npy'
data = np.load(fname)
# plt.imshow(data[:,:,0])
# plt.show()

['baseline', 'csd']
(128, 128, 5000)


In [5]:
# compute SVD on data 
u, Vs = svd_gpu(data.transpose(2,0,1), fname)


Data in:  (5000, 128, 128)
 Data going into GPU SVD:  (5000, 16384)
loading from disk
Vs:  (5000, 128, 128)
u:  (5000, 5000)
DONE


In [79]:
# denoise the data usinv SVD up to fixed rank
rank=50
denoised = svd_denoise(rank, u, Vs, fname)


In [11]:
# compute drift from fixed point backwards and forwadr in time

root_dirs = ['/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_A1',
             '/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_B4',
             '/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort3_4',
             '/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort4_1',
             '/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort5_2',
             '/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort6_DC2']

ctr_dir = 0
for root_dir in root_dirs:             
    fnames = os.listdir(root_dir)
    print (fnames)
    clrs=['blue','red']
    ctr=0
    
    fname_out = root_dir+'/all_frame_diff.png'
    
    print (fname_out)
    if os.path.exists(fname_out)==False:
        fig=plt.figure(figsize=(20,15))
        
        for fname in fnames:
            denoised = np.load(root_dir+'/'+fname+'/denoised.npy')

            # loop
            n_frames = 450
            trigger_frames = np.arange(n_frames,5000-n_frames,1)
            traces = []
            t = np.arange(-n_frames,n_frames,1)/150.
            if True:
                for trigger_frame in trigger_frames:
                    trigger_img = denoised[:,:,trigger_frame]

                    # compute difference between current frame and the next n_frames
                    diff = denoised[:,:,trigger_frame-n_frames:trigger_frame+n_frames].transpose(2,0,1)-trigger_img
                    sqrDiff = (diff**2).sum(1)

                    res = np.sqrt(sqrDiff.sum(axis=1))
                    traces.append(res)

            # broadcast version
            else:
                print (trigger_frames[None]-n_frames)
                print (trigger_frames[None]+n_frames)
                diff = denoised[:,:,trigger_frames[None]-n_frames:
                                    trigger_frames[None]+n_frames].transpose(2,0,1)-trigger_img
                print (diff.shape)

                sqrDiff = (diff**2).sum(1)

                res = np.sqrt(sqrDiff.sum(axis=1))

            # 
            traces=np.array(traces)
            mean = np.mean(traces,axis=0)
            std = np.std(traces,axis=0)

            print (t.shape, mean.shape, std.shape)
            plt.errorbar(t, mean, std, linestyle='None', marker='^', c=clrs[ctr], alpha=.5)

            ctr+=1


        #for k in range 
        plt.xlabel("Time (sec)", fontsize=20)
        plt.suptitle(root_dir+'\nblue: '+fnames[0]+"    red: "+fnames[1],fontsize=20)
        plt.savefig(fname_out)

        plt.close()
    else:
        print ("already computed")
    ctr_dir+=1

['baseline', '0.png', 'csd']
/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_A1/0.png
already computed
['1.png', 'baseline', 'csd']
/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_B4/1.png
already computed
['baseline', '2.png', 'control']
/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort3_4/2.png
already computed
['baseline', '3.png', 'control']
/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort4_1/3.png
already computed
['4.png', 'baseline', 'csd']
/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort5_2/4.png
already computed
['5.png', 'baseline', 'csd']
/media/cat/4TBSSD/Dropbox/Alex/Alexander McGirr - iGluSnFR_CSD_Cohort6_DC2/5.png
already computed


In [ ]:
# plot the drift away from specific frames
ctr_dir = 0
for root_dir in root_dirs:             
    fnames = os.listdir(root_dir)
    print (fnames)
    clrs=['blue','red']
    ctr=0
    
    fname_out = root_dir+'/all_frame_diff.png'
    
    print (fname_out)
    if os.path.exists(fname_out)==False:
        fig=plt.figure(figsize=(20,15))
        
        for fname in fnames:
            denoised = np.load(root_dir+'/'+fname+'/denoised.npy')

            # loop
            n_frames = 450
            trigger_frames = np.arange(n_frames,5000-n_frames,1)
            traces = []
            t = np.arange(-n_frames,n_frames,1)/150.
            if True:
                for trigger_frame in trigger_frames:
                    trigger_img = denoised[:,:,trigger_frame]

                    # compute difference between current frame and the next n_frames
                    diff = denoised[:,:,trigger_frame-n_frames:trigger_frame+n_frames].transpose(2,0,1)-trigger_img
                    sqrDiff = (diff**2).sum(1)

                    res = np.sqrt(sqrDiff.sum(axis=1))
                    traces.append(res)

            # broadcast version
            else:
                print (trigger_frames[None]-n_frames)
                print (trigger_frames[None]+n_frames)
                diff = denoised[:,:,trigger_frames[None]-n_frames:
                                    trigger_frames[None]+n_frames].transpose(2,0,1)-trigger_img
                print (diff.shape)

                sqrDiff = (diff**2).sum(1)

                res = np.sqrt(sqrDiff.sum(axis=1))

            # 
            traces=np.array(traces)
            mean = np.mean(traces,axis=0)
            std = np.std(traces,axis=0)

            print (t.shape, mean.shape, std.shape)
            plt.errorbar(t, mean, std, linestyle='None', marker='^', c=clrs[ctr], alpha=.5)

            ctr+=1


        #for k in range 
        plt.xlabel("Time (sec)", fontsize=20)
        plt.suptitle(root_dir+'\nblue: '+fnames[0]+"    red: "+fnames[1],fontsize=20)
        plt.savefig(fname_out)

        plt.close()
    else:
        print ("already computed")
    ctr_dir+=1


In [7]:
Vs = (v*s).transpose(1,0).reshape(-1,128,128)
np.save(root_dir+'Vs.npy',Vs)


In [5]:
# compute SVD on dataset to get temporal and spatial components; usually 5-10k frames is more than enough

# use GPU

if True:
    data_in = data_dff
    print (data_in.shape)
    data_in = np.float32(data_in)
    print (data_in.mean(0).shape)

    data_in = data_in.reshape(data_in.shape[0],-1)
    print (" Data going into GPU SVD: ", data_in.shape)

    a = torch.from_numpy(np.float32(data_in)).float().to(device)
    print ("Input array size: ", a.shape)
    print ("...starting GPU-svd")
    u, s, v = torch.svd(a)

# use CPU
else:
    import sklearn
    import sklearn.utils.extmath
    data_in = data_dff.reshape(data.shape[0],-1)
    print (" Data going into CPU SVD: ", data_in.shape)

    u, s, v = sklearn.utils.extmath.randomized_svd(data_in, 500, n_oversamples=10, n_iter='auto', 
                                                    power_iteration_normalizer='auto', transpose='auto', 
                                                    flip_sign=True, random_state=0)

print ("u: ", u.shape)
print ("s: ", s.shape)
print ("v: ", v.shape)

# save U x s and V x s matrices correct shapes
#Us = u 
#print ("us: ", Us.shape)

Vs = (v*s).transpose(1,0).reshape(-1,128,128)
print ("vs: ", Vs.shape)
    
torch.save(Vs,root_dir+'Vs_gpu.npy')
torch.save(u,root_dir+'Us_gpu.npy')


(39145, 128, 128)
(128, 128)
 Data going into GPU SVD:  (39145, 16384)
Input array size:  torch.Size([39145, 16384])
...starting GPU-svd
u:  torch.Size([39145, 16384])
s:  torch.Size([16384])
v:  torch.Size([16384, 16384])
vs:  torch.Size([16384, 128, 128])


In [8]:
# reload components and save componnents to disk
np.save(root_dir + 'Vs_dff_svd500.npy',torch.load(root_dir+'Vs_gpu.npy')[:500].cpu().data.numpy())
np.save(root_dir + 'Us_dff_svd500.npy',torch.load(root_dir+'Us_gpu.npy')[:,:500].cpu().data.numpy())

In [7]:
Vs = torch.load('/home/cat/Vs.npy')
Us = torch.load('/home/cat/Us.npy')

In [8]:
print (Vs.shape)
print (Us.shape)

torch.Size([16384, 128, 128])
torch.Size([39145, 16384])


In [20]:
np.save('/home/cat/Vs_dff.npy',Vs.cpu().data.numpy())
np.save('/home/cat/Us_dff.npy',Us.cpu().data.numpy())


In [16]:
u = np.load('/media/cat/10TB/in_vivo/tim/yuki/IA2/tif_files/IA2pm_Feb1_30Hz/u.npy')
print (u.shape)

Vs = np.load('/media/cat/10TB/in_vivo/tim/yuki/IA2/tif_files/IA2pm_Feb1_30Hz/Vs.npy')
print (Vs.shape)


(39149, 16384)
(16384, 128, 128)


In [18]:
plt.imshow(Vs[1])
#plt.plot(u[:,:5])
plt.show()

In [7]:
data = np.load('/home/cat/Uc.npy')
print (data.shape)



(679, 586, 500)


In [10]:
plt.imshow(data[:,:,2])
plt.show()

In [18]:
np.save(fname[:-4]+"_Uc.npy", Vs.cpu())
np.save(fname[:-4]+"_Uc.npy", Vs.cpu().data.numpy())
np.save(fname[:-4]+"_Vc.npy", Us.transpose(1,0).cpu().data.numpy())
#np.save(fname[:-4]+"_Vc.npy", Us.transpose(1,0).cpu().data.numpy())


RuntimeError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 11.91 GiB total capacity; 9.17 GiB already allocated; 471.88 MiB free; 960.00 KiB cached)

In [23]:
fname = '/home/cat/maskwarp.mat'
data = sio.loadmat(fname)
print (data)
img = data['maskwarp']
plt.imshow(img)
plt.show()

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Jan 26 13:39:29 2019', '__version__': '1.0', '__globals__': [], 'maskwarp': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}


In [3]:
#data1 = np.load('/media/cat/10TB/in_vivo/tim/yuki/IA1/tif_files/IA1am_May10_Week5_30Hz/IA1am_May10_Week5_30Hz.npy')
#print (data1.shape)

data2= np.load('/media/cat/10TB/in_vivo/tim/yuki/AI3/tif_files/AI3_Dec9am_15Hz_8x8/AI3_Dec9am_15Hz_8x8.npy')
print (data2.shape)

(20001, 128, 128)


In [4]:
plt.imshow(data2[1000])
plt.show()

In [126]:
rot = np.float32([[3.41, 0.07, 0],
       [-0.07, 3.41, 0],
       [47.87, 73.70, 1]])
print (rot)


[[ 3.410e+00  7.000e-02  0.000e+00]
 [-7.000e-02  3.410e+00  0.000e+00]
 [ 4.787e+01  7.370e+01  1.000e+00]]


In [127]:
#rot = np.load('/home/cat/tform.npy')
print (rot)

#fromm numpy.linalg import inv
rot_inv = np.linalg.inv(rot[:3,:3])
print (rot_inv)

[[ 3.410e+00  7.000e-02  0.000e+00]
 [-7.000e-02  3.410e+00  0.000e+00]
 [ 4.787e+01  7.370e+01  1.000e+00]]
[[ 2.9313159e-01 -6.0173641e-03  0.0000000e+00]
 [ 6.0173641e-03  2.9313159e-01 -0.0000000e+00]
 [-1.4475689e+01 -2.1315746e+01  1.0000000e+00]]


In [9]:
rot_inv = np.float32([
    [ 3.2845,    0.0535,         0],
    [-0.0535,    3.2845,         0],
    [57.0638,   81.2616,    1.0000]    
])

In [4]:
#convert .tiff to npy. 
import os
from skimage import io, exposure
import numpy as np

fname = '/media/cat/4TB_ephys/1TB/alex/Alexander McGirr - iGluSnFR_CSD_Cohort6_DC2/Cohort6_DC2/24Hrs/Spontaneous_1iso_150Hz_50000fr.tif'

if os.path.exists(fname[:-4]+".npy")==False:

    images = io.imread(fname, plugin='tifffile')
    print (images.shape)
    np.save(fname[:-4]+".npy", images)

    images = np.float32(np.load(fname[:-4]+".npy"))
else:
    images = np.load(fname[:-4]+".npy")
    
print (images.shape)

(50001, 128, 128)
(50001, 128, 128)


In [9]:
# compute SVD on data 
root_dir = '/media/cat/2TB/alex/Alexander McGirr - iGluSnFR_CSD_Cohort6_DC2/Cohort6_DC2/24Hrs/'

u = np.load(root_dir+'u_gpu.npy', allow_pickle=True)
s = np.load(root_dir+'s_gpu.npy', allow_pickle=True)
v = np.load(root_dir+'v_gpu.npy', allow_pickle=True)
print (u)

u = np.load('/media/cat/2TB/alex/Alexander McGirr - iGluSnFR_CSD_Cohort6_DC2/Cohort6_DC2/Baseline/Us_gpu.npy', allow_pickle=True)

print ("u: ", u.shape)
print ("s: ", s.shape)
print ("v: ", v.shape)

# scale temporal component by singular vals
#Us = (u*s).transpose(1,0).reshape(-1,128,128)

torch.save(v,root_dir+'v_gpu.npy')
torch.save(s,root_dir+'s_gpu.npy')
torch.save(u,root_dir+'u_gpu.npy')


119547037146038801333356


AttributeError: 'int' object has no attribute 'shape'